In [368]:
import math
import numpy as np
import pandas as pd
from scipy.stats import norm

# Question1

In [369]:
N = 60
T = 5
dt = T / N
r00 = 0.053
pi = 0.5

## (a)

In [370]:
# Short rate tree
short_rate_tree = np.zeros((N+1,N+1))
for i in range(N+1):  # time steps count
    for j in range(i+1):  # up states count
        short_rate_tree[i,j] =  r00*np.exp(0.25*(2*j-i)*np.sqrt(dt))
short_rate_tree

array([[5.30000000e-02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.93098141e-02, 5.69663474e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.58765616e-02, 5.30000000e-02, 6.12295233e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [8.06140255e-04, 9.31312897e-04, 1.07592160e-03, ...,
        3.48450531e+00, 0.00000000e+00, 0.00000000e+00],
       [7.50011813e-04, 8.66469166e-04, 1.00100932e-03, ...,
        3.24189262e+00, 3.74527434e+00, 0.00000000e+00],
       [6.97791379e-04, 8.06140255e-04, 9.31312897e-04, ...,
        3.01617212e+00, 3.48450531e+00, 4.02555848e+00]])

In [371]:
# Arrow-Debreu states tree
AD_tree = np.zeros((N+1,N+1))
AD_tree[0, 0] = 1
for i in range(1, N+1):
    for j in range(i+1):
        if j == 0:
            AD_tree[i, 0] = (1 - pi) * AD_tree[i - 1, 0] / (1 + short_rate_tree[i - 1, 0] * dt)
        elif i == j:
            AD_tree[i, j] = pi * AD_tree[i - 1, j - 1] / (1 + short_rate_tree[i - 1, j - 1] * dt)
        elif i > j:
            AD_tree[i, j] = (1 - pi) * AD_tree[i - 1, j]  / (1 + short_rate_tree[i - 1, j] * dt) +\
            pi * AD_tree[i - 1, j - 1] / (1 + short_rate_tree[i - 1, j - 1] * dt)
AD_tree

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.97801377e-01, 4.97801377e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.47882104e-01, 4.95606795e-01, 2.47724691e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.25958024e-18, 1.88662952e-16, 5.36503887e-15, ...,
        9.68198380e-20, 0.00000000e+00, 0.00000000e+00],
       [1.62968064e-18, 9.59538361e-17, 2.77660310e-15, ...,
        3.22570857e-18, 3.75161502e-20, 0.00000000e+00],
       [8.14789396e-19, 4.87882435e-17, 1.43615920e-15, ...,
        5.44509580e-17, 1.28410246e-18, 1.42961562e-20]])

## (b)

In [372]:
# 1 year caplets
reset_date = 9
payoff_date = 12
bond_tree = np.zeros((4,payoff_date+1))
bond_tree[3,:] = 1

for i in range(2,-1,-1):
    for j in range(payoff_date+i-2):
        bond_tree[i,j] = bond_tree[i+1,j]/(1+short_rate_tree[payoff_date+i-3,j]*dt)*(1-pi)+ bond_tree[i+1,j+1]/(1+short_rate_tree[payoff_date+i-3,j]*dt)*pi
payoff = ((1/bond_tree[0,:reset_date+1]-1)/0.25-0.05)*0.25
payoff[payoff<0] = 0
payoff_pv = payoff*bond_tree[0,:reset_date+1]

payoff_tree = np.zeros((reset_date+1,reset_date+1))
payoff_tree[reset_date,:] = payoff_pv
for i in range(reset_date-1,-1,-1):
    for j in range(i+1):
        payoff_tree[i,j] = payoff_tree[i+1,j]/(1+short_rate_tree[i,j]*dt)*(1-pi)+ payoff_tree[i+1,j+1]/(1+short_rate_tree[i,j]*dt)*pi
price1 = payoff_tree[0,0]
print("The value of 1 year caplets",round(payoff_tree[0,0], 8))

The value of 1 year caplets 0.00168256


In [373]:
# 3 year caplets
reset_date = 33
payoff_date = 36
bond_tree = np.zeros((4,payoff_date+1))
bond_tree[3,:] = 1

for i in range(2,-1,-1):
    for j in range(payoff_date+i-2):
        bond_tree[i,j] = bond_tree[i+1,j]/(1+short_rate_tree[payoff_date+i-3,j]*dt)*(1-pi)+ bond_tree[i+1,j+1]/(1+short_rate_tree[payoff_date+i-3,j]*dt)*pi
payoff = ((1/bond_tree[0,:reset_date+1]-1)/0.25-0.05)*0.25
payoff[payoff<0] = 0
payoff_pv = payoff*bond_tree[0,:reset_date+1]

payoff_tree = np.zeros((reset_date+1,reset_date+1))
payoff_tree[reset_date,:] = payoff_pv
for i in range(reset_date-1,-1,-1):
    for j in range(i+1):
        payoff_tree[i,j] = payoff_tree[i+1,j]/(1+short_rate_tree[i,j]*dt)*(1-pi)+ payoff_tree[i+1,j+1]/(1+short_rate_tree[i,j]*dt)*pi
price3 = payoff_tree[0,0]
print("The value of 3 year caplets",round(payoff_tree[0,0], 8))

The value of 3 year caplets 0.00273085


In [374]:
# 5 year caplets
reset_date =57
payoff_date = 60
bond_tree = np.zeros((4,payoff_date+1))
bond_tree[3,:] = 1

for i in range(2,-1,-1):
    for j in range(payoff_date+i-2):
        bond_tree[i,j] = bond_tree[i+1,j]/(1+short_rate_tree[payoff_date+i-3,j]*dt)*(1-pi)+ bond_tree[i+1,j+1]/(1+short_rate_tree[payoff_date+i-3,j]*dt)*pi
payoff = ((1/bond_tree[0,:reset_date+1]-1)/0.25-0.05)*0.25
payoff[payoff<0] = 0
payoff_pv = payoff*bond_tree[0,:reset_date+1]

payoff_tree = np.zeros((reset_date+1,reset_date+1))
payoff_tree[reset_date,:] = payoff_pv
for i in range(reset_date-1,-1,-1):
    for j in range(i+1):
        payoff_tree[i,j] = payoff_tree[i+1,j]/(1+short_rate_tree[i,j]*dt)*(1-pi)+ payoff_tree[i+1,j+1]/(1+short_rate_tree[i,j]*dt)*pi
price5 = payoff_tree[0,0]
print("The value of 5 year caplets",round(payoff_tree[0,0], 8))

The value of 5 year caplets 0.00315075


## (c)

In [375]:
P1 = sum(AD_tree[12,:])
P3 = sum(AD_tree[36,:])
P5 = sum(AD_tree[60,:])
fr1 = (sum(AD_tree[9,:])/P1-1)/0.25
fr3 = (sum(AD_tree[33,:])/P3-1)/0.25
fr5 = (sum(AD_tree[57,:])/P5-1)/0.25
T1 = 0.75
T3 = 2.75
T5 = 4.75
K = 0.05
tau = 0.25
sigma = np.linspace(0.000001,1,1000000)

In [376]:
# 1 year caplets
d1 = (np.log(fr1/K)+0.5*sigma**2*T1)/sigma/np.sqrt(T1)
d2 = (np.log(fr1/K)-0.5*sigma**2*T1)/sigma/np.sqrt(T1)
Black_price1 = tau*P1*(fr1*norm.cdf(d1)-K*norm.cdf(d2))
abs_dif1 = np.abs(Black_price1 - price1)
sigma1 = sigma[abs_dif1 == np.min(abs_dif1)][0]
print("The implied Black volatilities of the forward rates F(0; 0.75, 1) = ", round(sigma1, 6))

The implied Black volatilities of the forward rates F(0; 0.75, 1) =  0.246716


In [377]:
# 3 year caplets
d1 = (np.log(fr3/K)+0.5*sigma**2*T3)/sigma/np.sqrt(T3)
d2 = (np.log(fr3/K)-0.5*sigma**2*T3)/sigma/np.sqrt(T3)
Black_price3 = tau*P3*(fr3*norm.cdf(d1)-K*norm.cdf(d2))
abs_dif3 = np.abs(Black_price3 - price3)
sigma3 = sigma[abs_dif3 == np.min(abs_dif3)][0]
print("The implied Black volatilities of the forward rates F(0; 2.75, 3) = ", round(sigma3, 6))

The implied Black volatilities of the forward rates F(0; 2.75, 3) =  0.248671


In [378]:
# 5 year caplets
d1 = (np.log(fr5/K)+0.5*sigma**2*T5)/sigma/np.sqrt(T5)
d2 = (np.log(fr5/K)-0.5*sigma**2*T5)/sigma/np.sqrt(T5)
Black_price5 = tau*P5*(fr5*norm.cdf(d1)-K*norm.cdf(d2))
abs_dif5 = np.abs(Black_price5 - price5)
sigma5 = sigma[abs_dif5 == np.min(abs_dif5)][0]
print("The implied Black volatilities of the forward rates F(0; 4.75, 5) = ", round(sigma5, 6))

The implied Black volatilities of the forward rates F(0; 4.75, 5) =  0.246796


## (d)

In [379]:
K = 0.055
maturity_date = 36
swap_date = [24, 18, 12, 6]
short_rate_chunk = short_rate_tree[maturity_date:,:]
cashflow_value = np.zeros_like(short_rate_chunk)
for sd in swap_date:
    n = sum(short_rate_chunk[sd,:]>0)
    bond_tree = np.zeros((7,n))
    bond_tree[6,:] = 1
    for i in range(5, -1, -1):
        for j in range(n+i-6):
            bond_tree[i,j] = bond_tree[i+1,j]/(1+short_rate_chunk[sd+i-6,j]*dt)*(1-pi)+ bond_tree[i+1,j+1]/(1+short_rate_chunk[sd+i-6,j]*dt)*pi
    cashflow_value[sd-6,:sd+maturity_date-5] = ((1/bond_tree[0,:sd+maturity_date-5]-1)/0.5-K)*bond_tree[0,:sd+maturity_date-5]*0.5
for i in range(23,-1,-1):
    for j in range(maturity_date+i+1):
        cashflow_value[i,j] += cashflow_value[i+1,j]/(1+short_rate_chunk[i,j]*dt)*(1-pi) + cashflow_value[i+1,j+1]/(1+short_rate_chunk[i,j]*dt)*pi
swaption_value = np.zeros((maturity_date+1,maturity_date+1))
swaption_value[maturity_date,:] = cashflow_value[0,:maturity_date+1]
swaption_value[swaption_value<0] = 0
for i in range(maturity_date-1,-1,-1):
    for j in range(i+1):
        swaption_value[i,j] = swaption_value[i+1,j]/(1+short_rate_tree[i,j]*dt)*(1-pi)+swaption_value[i+1,j+1]/(1+short_rate_tree[i,j]*dt)*pi
swaption_price = swaption_value[0,0]
print("The swaption value is ",round(swaption_value[0,0],8))

The swaption value is  0.01848437


## (e)

In [380]:
K = 0.055
T = 3
swap_dt = 0.5
maturity_date = 36
Fixed0 = swap_dt*np.sum(AD_tree[[maturity_date+6*i for i in range(1,5)],:])
S_T_Tn = (sum(AD_tree[maturity_date,:])-sum(AD_tree[60,:]))/Fixed0
sigma = np.linspace(0.000001,1,1000000)

In [381]:
d1 = (np.log(S_T_Tn/K)+0.5*sigma**2*T)/sigma/np.sqrt(T)
d2 = (np.log(S_T_Tn/K)-0.5*sigma**2*T)/sigma/np.sqrt(T)
Black_price_swqption = Fixed0*(S_T_Tn*norm.cdf(d1)-K*norm.cdf(d2))
abs_dif = np.abs(Black_price_swqption - swaption_price)
sigma_swap = sigma[abs_dif == np.min(abs_dif)][0]
print("The Black implied volatility of the swap rate from part d) = ", round(sigma_swap, 6))

The Black implied volatility of the swap rate from part d) =  0.247913


## (f)

In [382]:
K = 0.055
maturity_date = 6
value_at_maturity = np.zeros((maturity_date+1,))
swap_date = [54, 48, 42, 36,30,24,18,12,6]
short_rate_chunk = short_rate_tree[maturity_date:,:]
cashflow_value = np.zeros_like(short_rate_chunk)
for sd in swap_date:
    n = sum(short_rate_chunk[sd,:]>0)
    bond_tree = np.zeros((7,n))
    bond_tree[6,:] = 1
    for i in range(5, -1, -1):
        for j in range(n+i-6):
            bond_tree[i,j] = bond_tree[i+1,j]/(1+short_rate_chunk[sd+i-6,j]*dt)*(1-pi)+ bond_tree[i+1,j+1]/(1+short_rate_chunk[sd+i-6,j]*dt)*pi
    cashflow_value[sd-6,:sd+maturity_date-5] = ((1/bond_tree[0,:sd+maturity_date-5]-1)/0.5-K)*bond_tree[0,:sd+maturity_date-5]*0.5
for i in range(53,-1,-1):
    for j in range(maturity_date+i+1):
        cashflow_value[i,j] += cashflow_value[i+1,j]/(1+short_rate_chunk[i,j]*dt)*(1-pi) + cashflow_value[i+1,j+1]/(1+short_rate_chunk[i,j]*dt)*pi
exercise_date = [24,18,12,6,0]
not_exercise_value = cashflow_value[32,:37]
not_exercise_value[not_exercise_value<0]=0
for ed in exercise_date:
    exercise_value = cashflow_value[ed,:ed+maturity_date+1]
    for i in range(ed+5,ed-1,-1):
        short_rate = short_rate_tree[maturity_date+i,:maturity_date+i+1]
        not_exercise_value = not_exercise_value[:maturity_date+i+1]/(1+short_rate*dt)*(1-pi)+not_exercise_value[1:]/(1+short_rate*dt)*pi
    not_exercise_value = np.maximum(not_exercise_value,exercise_value)         
swaption_value = np.zeros((maturity_date+1,maturity_date+1))
swaption_value[maturity_date,:] = not_exercise_value
swaption_value[swaption_value<0] = 0
for i in range(maturity_date-1,-1,-1):
    for j in range(i+1):
        swaption_value[i,j] = swaption_value[i+1,j]/(1+short_rate_tree[i,j]*dt)*(1-pi)+swaption_value[i+1,j+1]/(1+short_rate_tree[i,j]*dt)*pi
Bermudan_swaption_price = swaption_value[0,0]
print("The Bermudan swaption value is ",round(swaption_value[0,0],8))

The Bermudan swaption value is  0.03037217


# Question3

## (a)

In [383]:
r0 = 0.053
theta = 0.005
k = 0.1
sigma = 0.025
t = 0

def A(t, T, theta, sigma, k):
    B_value = B(t, T, k)
    first_term = (theta - (sigma**2 / (2 * (k**2)))) * (B_value - (T - t))
    second_term = (sigma**2 / (4 * k)) * B_value**2
    return math.exp(first_term - second_term)

def B(t, T, k):
    return (1 - math.exp(-k * (T - t))) / k

def P(t, T, r0, theta, sigma, k):
    return A(t, T, theta, sigma, k) * math.exp(-B(t, T, k) * r0)

def implied_rate(P_value, T):
    return -math.log(P_value) / T

maturities = [1, 5, 10, 20, 30]
bond_prices = [P(t, T, r0, theta, sigma, k) for T in maturities]
implied_rates = [implied_rate(P_value, T) for P_value, T in zip(bond_prices, maturities)]

ZCB_value = pd.DataFrame()
ZCB_value["maturity"] = maturities
ZCB_value["Bond_Prices"] = bond_prices
ZCB_value["Implied_Rates"] = implied_rates
print(ZCB_value)

   maturity  Bond_Prices  Implied_Rates
0         1     0.950677       0.050581
1         5     0.814839       0.040953
2        10     0.740160       0.030089
3        20     0.758004       0.013853
4        30     0.898865       0.003554


## (b)

The bond price decreases as maturity increases up to 10 years. However, at the 20-year maturity, the bond price increases compared to the 10-year maturity, and, at the 30-year maturity, the bond price again increases significantly. It implies that the rate after 10 years is negative, which is an unusual feature and might don;t match typical market behaviors.

## (c)

In [384]:
r0_values = [0.052, 0.054]
implied_rates_scenarios = []

for r in r0_values:
    rates = [implied_rate(P(t, T, r, theta, sigma, k), T) for T in maturities]
    implied_rates_scenarios.append(rates)

Implied_Rates = pd.DataFrame()
Implied_Rates["maturity"] = maturities
Implied_Rates["Implied Rates for r0=0.052"] = implied_rates_scenarios[0]
Implied_Rates["Implied Rates for r0=0.054"] = implied_rates_scenarios[1]
print(Implied_Rates)

   maturity  Implied Rates for r0=0.052  Implied Rates for r0=0.054
0         1                    0.049630                    0.051533
1         5                    0.040166                    0.041740
2        10                    0.029457                    0.030721
3        20                    0.013421                    0.014286
4        30                    0.003237                    0.003871


In [386]:
rates_matrix = np.array([implied_rates, implied_rates_scenarios[0], implied_rates_scenarios[1]])
correlations = np.corrcoef(rates_matrix)
cor = pd.DataFrame(correlations, columns=['r0','down state','up state'],index=['r0','down state','up state'])
cor

,r0,down state,up state
r0,1.000000,0.999999,1.000000
down state,0.999999,1.000000,0.999998
up state,1.000000,0.999998,1.000000


The correlations are very high, which means the yield curve moves in a nearly perfect parallel pattern when $ r_t $ changes. This pattern is not very realistic, for in real world, yield curves usually exhibit several different type of movement. 